# ETF flow decomposition 🌊

> Nearest neighbor functions, etc.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp flow_decomposition

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree
from numpy.typing import NDArray

In [ ]:
# | export


def get_times(df: pd.DataFrame) -> NDArray[np.datetime64]:
    if df.index.values.dtype == "datetime64[ns]":
        return df.index.values.reshape(-1, 1)
    else:
        return df["datetime"].values.reshape(-1, 1)


def str_to_time(time: str, convert_to: str) -> int:
    return pd.Timedelta(time) / pd.Timedelta(1, unit=convert_to)


# better way to compose with partial (i.e without lambda)? 
str_to_nanoseconds = lambda x: int(str_to_time(x, convert_to="ns"))

In [ ]:
# | export


def add_neighbors(
    etf_executions: pd.DataFrame,
    equity_executions: pd.DataFrame,
    tolerance: str | list[str],
):
    """Annotate the etf execution dataframe with the indices of the neighbouring equity executions.
    Note: Building the KDTree on the equity dataframe.
    """
    etf_executions_times = get_times(etf_executions)
    equity_executions_times = get_times(equity_executions)
    equity_tree = KDTree(equity_executions_times, metric="l1")

    def _add_neighbors_col(etf_executions, tolerance_str):
        tolerance_in_nanoseconds = str_to_nanoseconds(tolerance_str)
        etf_executions[f"_{tolerance_str}_neighbors"] = equity_tree.query_radius(etf_executions_times, r=tolerance_in_nanoseconds)

        etf_executions[f"_{tolerance_str}_neighbors"] = etf_executions[f"_{tolerance_str}_neighbors"].apply(
            lambda x: None if x.size == 0 else x
        )

        # etf_executions[f"_{tolerance_str}_neighbors"] = pd.Series(etf_executions[f"_{tolerance_str}_neighbors"], dtype="Sparse[object]")
        return None

    if isinstance(tolerance, str):
        _add_neighbors_col(etf_executions, tolerance)

    elif isinstance(tolerance, list):
        for tolerance in tolerance:
            _add_neighbors_col(etf_executions, tolerance)
    else:
        raise ValueError("tolerance_str must be a string or a list of strings")


def drop_all_neighbor_cols(df: pd.DataFrame):
    "Drop neighbor columns inplace."
    neighbor_column_names = df.filter(regex="neighbors").columns
    df.drop(columns=neighbor_column_names, inplace=True)
    return None

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()